In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from datetime import date, timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import datetime
from datetime import date, timedelta

<h1>Data preparation and pre-processing</h1>

<h2>Functions</h2>

In [2]:
### Functions

def load_data(path):
    return (pd.read_csv(path))

def prepare_data(df_transactions, df_players):
    
    df_transactions.columns = ["x"]
    df_transactions = df_transactions["x"].str.split(';', expand = True)
    df_transactions.columns = ["player_id", "transaction_date", "product", "transaction_type", "amount", "count"]
    
    #change types
    df_transactions['player_id'] = df_transactions['player_id'].astype('int32')
    df_transactions['transaction_date'] = df_transactions['transaction_date'].astype('datetime64')
    df_transactions['product'] = df_transactions['product'].astype('string')
    df_transactions['transaction_type'] = df_transactions['transaction_type'].astype('string')
    df_transactions['amount'] = df_transactions['amount'].astype('float32')
    df_transactions['count'] = df_transactions['count'].astype('int32')
    
    df_players.columns = ["player_id", "birth_date", "city", "registration_date", "registration_hour", "is_opt_out", "registration_terminal"]
    
    #change types
    df_players['birth_date'] = df_players['birth_date'].astype('datetime64')
    df_players['city'] = df_players['city'].astype('string')
    df_players['registration_date'] = df_players['registration_date'].astype('datetime64')
    df_players['registration_hour'] = df_players['registration_hour'].astype('float32')
    
    return df_transactions, df_players

def get_new_features(df):
    
    # Prepare for calculating new features later 
    
    # Time since registration
    df['time_since_registration'] = (df['transaction_date'] - df['registration_date']).dt.total_seconds()
    
    # Frequency and monetary value
    df['prev_amount'] = df.groupby('player_id')['amount'].shift(1)
    df['prev_amount'].fillna(df['amount'], inplace = True)
    
    df['frequency'] = (df.groupby('player_id')['transaction_date'].cumcount() + 1)
    
    df['monetary_value'] = df.groupby('player_id')['prev_amount'].cumsum()
    
    df.drop(['prev_amount'], axis = 1, inplace = True)

    
    return df

def clean_data(df):
    
    len_before_clean = len(df)
    
    # For some rows transaction_date is before registration_date which is not possible
    df = df.drop(df[df['transaction_date'] < df['registration_date']].index)
    
    # We have some null values (very small amount so best option is to delete)
    df = df.dropna()
    
    len_after_clean = len(df)
    
    print("Removed ", (1 - len_after_clean / len_before_clean) * 100, "% of data.", sep = '')
    
    return df

<h2>Load dataset and prepare to calculate new features</h2>

In [3]:
df_transactions = load_data("dataset/zadatak-lite.csv")
df_players = load_data("dataset/igraci.csv")

df_transactions, df_players = prepare_data(df_transactions, df_players)

df = pd.merge(df_transactions, df_players, on = 'player_id', how = 'left')

df = clean_data(df)

df.sort_values(by = 'transaction_date', inplace = True)

Removed 5.001921881014326% of data.


In [4]:
df = get_new_features(df)

In [5]:
print(df['transaction_date'].min(), df['transaction_date'].max(), sep = '\n')

2022-04-01 00:00:00
2022-12-31 00:00:00


In [6]:
start_date = date(2022, 5, 1)
end_date = date(2022, 11, 30)
date_range = [start_date + timedelta(days=x) for x in range((end_date-start_date).days + 1)]

new_df = pd.DataFrame(columns = ['player_id', 'date', 'bo_count', 'casino_count', 'pp_count', 'sport_count', 'vb_count',
       'vdr_count', 'frequency', 'monetary_value', 'profit', 'deposit',
       'last_active', 'player_age', 'time_since_registration', 'is_opt_out',
       'churn'])

for d in date_range:
    print(d)
    d = pd.to_datetime(d)

    last_30_days_df = df[(df['transaction_date'] >= d - timedelta(days = 30)) & (df['transaction_date'] < d)]
    
    # Count by product
    # Group by player_id, product
    product_count_by_player = last_30_days_df.groupby(['player_id','product'])['count'].sum().reset_index()
    product_count_by_player = product_count_by_player.pivot(index = 'player_id', columns = 'product', values = 'count')
    # Fillna with 0
    product_count_by_player.fillna(0, inplace=True)
    product_count_by_player.rename(columns = {'Sport': 'sport_count','Casino':'casino_count',
                                              'PaymentProvider':'pp_count','BusinessOwner':'bo_count',
                                              'VirtualBingo':'vb_count', 'VirtualDogRace': 'vdr_count'},
                                               inplace = True)
    
    # Frequency
    frequency_by_player = last_30_days_df.groupby(['player_id'])['frequency'].max() - last_30_days_df.groupby(['player_id'])['frequency'].min()
    
    frequency_by_player = frequency_by_player.to_frame()
    frequency_by_player = frequency_by_player.rename(columns={0: "frequency"})

    
    # Monetary value
    monetary_by_player = last_30_days_df.groupby(['player_id'])['monetary_value'].max() - last_30_days_df.groupby(['player_id'])['monetary_value'].min()
    
    monetary_by_player = monetary_by_player.to_frame()
    monetary_by_player = monetary_by_player.rename(columns={0: "monetary_value"})
    
    # Profit, Deposit
    transaction_amount_by_player = last_30_days_df.groupby(['player_id', 'transaction_type'])['amount'].sum().reset_index()
    transaction_amount_by_player = transaction_amount_by_player.pivot(index = 'player_id', columns = 'transaction_type', values = 'amount')
    transaction_amount_by_player.fillna(0, inplace = True)
    transaction_amount_by_player.rename(columns = {'Bonus': 'bonus', 'TicketWin': 'ticketwin','TicketPayin':'payin','Deposit':'deposit','Withdrawal':'withdrawal'}, inplace = True)
    profit = transaction_amount_by_player['ticketwin'] + transaction_amount_by_player['bonus'] - transaction_amount_by_player['payin']
    transaction_amount_by_player.insert(0, 'profit', profit)
    profit_by_player = transaction_amount_by_player.drop(columns = ['bonus', 'payin',
                                    'ticketwin', 'withdrawal', 'DepositCancel', 'TicketPayinCancel', 'TicketWinCancel'])
    
    # Last active
    last_active_by_player = (d - last_30_days_df.groupby(['player_id'])['transaction_date'].max())
    
    last_active_by_player = last_active_by_player.to_frame()
    last_active_by_player = last_active_by_player.rename(columns={'transaction_date': "last_active"})
    
    # Player age
    player_age = d - last_30_days_df.groupby(['player_id'])['birth_date'].max()
    
    player_age = player_age.to_frame()
    player_age = player_age.rename(columns={'birth_date': "player_age"})
    
    #Time since registration
    time_since_registration = d - last_30_days_df.groupby(['player_id'])['registration_date'].max()
    
    time_since_registration = time_since_registration.to_frame()
    time_since_registration = time_since_registration.rename(columns={'registration_date': "time_since_registration"})
    
    # Is opt out
    is_opt_out = last_30_days_df.groupby(['player_id'])['is_opt_out'].max()
    
    is_opt_out = is_opt_out.to_frame()
    is_opt_out = is_opt_out.rename(columns={0: "is_opt_out"})
    
    # Merge all features in one dataframe
    concat_df = pd.concat([product_count_by_player, frequency_by_player, monetary_by_player,
                          profit_by_player, last_active_by_player, player_age,
                          time_since_registration, is_opt_out], axis=1)
    
    # Fix for player_id column
    concat_df['player_id'] = concat_df.index
    
    # Churn
    
    concat_df['churn'] = concat_df['player_id'].isin(df[(df['transaction_date'] > d) & 
                                (df['transaction_date'] < d + timedelta(days = 30))]['player_id']).astype(int)
    
    concat_df['churn'] = concat_df['churn'].replace({0:1, 1:0})
    
    # Date
    concat_df['date'] = d
    
    # Append to complete dataframe
    
    new_df['frequency'] = new_df['frequency'].astype('int')
    new_df['is_opt_out'] = new_df['is_opt_out'].astype('int')
    new_df['player_id'] = new_df['player_id'].astype('int')
    new_df['churn'] = new_df['churn'].astype('int')
    
    new_df = pd.concat([new_df, concat_df], ignore_index = True)
    

new_df = new_df.drop('WithdrawalCancel', axis = 1)
new_df

2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-11
2022-06-12
2022-06-13
2022-06-14
2022-06-15
2022-06-16
2022-06-17
2022-06-18
2022-06-19
2022-06-20
2022-06-21
2022-06-22
2022-06-23
2022-06-24
2022-06-25
2022-06-26
2022-06-27
2022-06-28
2022-06-29
2022-06-30
2022-07-01
2022-07-02
2022-07-03
2022-07-04
2022-07-05
2022-07-06
2022-07-07
2022-07-08
2022-07-09
2022-07-10
2022-07-11
2022-07-12
2022-07-13
2022-07-14
2022-07-15
2022-07-16
2022-07-17
2022-07-18
2022-07-19
2022-07-20
2022-07-21
2022-07-22
2022-07-23
2022-07-24
2022-07-25
2022-07-26
2022-07-27
2022-07-28
2022-07-29
2022-07-30

,player_id,date,bo_count,casino_count,pp_count,sport_count,vb_count,vdr_count,frequency,monetary_value,profit,deposit,last_active,player_age,time_since_registration,is_opt_out,churn
0,2,2022-05-01,1.0,0.0,1.0,13.0,0.0,0.0,2,38.009998,-1.809998,1.810000,22 days,10168 days,2777 days,0,0
1,4,2022-05-01,11.0,16992.0,79.0,25.0,6153.0,0.0,167,2495.419922,-258.329956,257.429993,2 days,18747 days,2777 days,0,0
2,5,2022-05-01,2.0,19796.0,43.0,0.0,0.0,0.0,28,7263.290039,-411.770020,592.309998,6 days,15087 days,2777 days,0,0
3,8,2022-05-01,0.0,0.0,1.0,8.0,0.0,0.0,7,4.910000,-1.630000,0.900000,1 days,18619 days,2776 days,0,0
4,11,2022-05-01,1.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,4.520000,0.000000,23 days,9963 days,2776 days,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809655,30926,2022-11-30,0.0,0.0,1.0,11.0,0.0,0.0,4,156.360001,47.779999,0.000000,1 days,11952 days,2 days,0,0
1809656,30927,2022-11-30,0.0,45.0,1.0,0.0,0.0,0.0,2,1.670000,-1.130000,1.130000,2 days,6664 days,2 days,0,0
1809657,30929,2022-11-30,0.0,0.0,0.0,2.0,0.0,0.0,0,0.000000,-1.000000,0.000000,2 days,7610 days,2 days,0,1
1809658,30931,2022-11-30,0.0,0.0,3.0,0.0,19.0,0.0,2,3.670000,-2.990000,2.990000,1 days,12095 days,2 days,0,0


In [7]:
new_df

,player_id,date,bo_count,casino_count,pp_count,sport_count,vb_count,vdr_count,frequency,monetary_value,profit,deposit,last_active,player_age,time_since_registration,is_opt_out,churn
0,2,2022-05-01,1.0,0.0,1.0,13.0,0.0,0.0,2,38.009998,-1.809998,1.810000,22 days,10168 days,2777 days,0,0
1,4,2022-05-01,11.0,16992.0,79.0,25.0,6153.0,0.0,167,2495.419922,-258.329956,257.429993,2 days,18747 days,2777 days,0,0
2,5,2022-05-01,2.0,19796.0,43.0,0.0,0.0,0.0,28,7263.290039,-411.770020,592.309998,6 days,15087 days,2777 days,0,0
3,8,2022-05-01,0.0,0.0,1.0,8.0,0.0,0.0,7,4.910000,-1.630000,0.900000,1 days,18619 days,2776 days,0,0
4,11,2022-05-01,1.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,4.520000,0.000000,23 days,9963 days,2776 days,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809655,30926,2022-11-30,0.0,0.0,1.0,11.0,0.0,0.0,4,156.360001,47.779999,0.000000,1 days,11952 days,2 days,0,0
1809656,30927,2022-11-30,0.0,45.0,1.0,0.0,0.0,0.0,2,1.670000,-1.130000,1.130000,2 days,6664 days,2 days,0,0
1809657,30929,2022-11-30,0.0,0.0,0.0,2.0,0.0,0.0,0,0.000000,-1.000000,0.000000,2 days,7610 days,2 days,0,1
1809658,30931,2022-11-30,0.0,0.0,3.0,0.0,19.0,0.0,2,3.670000,-2.990000,2.990000,1 days,12095 days,2 days,0,0


In [8]:
# Save dataframe ready for model
tmp_df = new_df
tmp_df.to_csv('dataset/new_feature_dataset.csv')